#Install Packages

In [1]:
!unset GOOGLE_APPLICATION_CREDENTIALS
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


import google.auth
credentials, project = google.auth.default()
from google.cloud.bigquery import magics
magics.context.credentials = credentials


In [2]:
#!gsutil cp gs://vertex_sdk_llm_private_releases/SDK/google_cloud_aiplatform-1.25.dev20230413+language.models-py2.py3-none-any.whl .
#!pip install google_cloud_aiplatform-1.25.dev20230413+language.models-py2.py3-none-any.whl "shapely<2.0.0" --quiet
#upgrade to new endpoints as per request 7/7/2023
!pip install google-cloud-aiplatform --upgrade  --quiet

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

!pip install smart-open --quiet
!pip install kaggle --quiet

import IPython
print( 'restarting kernel...' )
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Vertex AI SDK version: 1.26.1
restarting kernel...


{'status': 'ok', 'restart': True}

In [1]:
%reload_ext autoreload
%autoreload 2

In [8]:
PROJECT_ID = "kallogjeri-project-345114" # @param {type:"string"} <---CHANGE THESE
KAGGLE_DATASET_USERNAME = "chaitanyakck" # @param {type:"string"} <---CHANGE THESE
KAGGLE_DATASET_NAME = "medical-text" # @param {type:"string"} <---CHANGE THESE
BUCKET_NAME = "aburdenko-llm-captstone" # @param {type:"string"} <---CHANGE THESE
#GOOGLE_APPLICATION_CREDENTIALS="/content/service_account.json"
#%env GOOGLE_APPLICATION_CREDENTIALS=$GOOGLE_APPLICATION_CREDENTIALS

WORKSPACE_HOME="/content" # @param {type:"string"} <---CHANGE THESE

import sys
IN_COLAB = 'google.colab' in sys.modules
%env IN_COLAB=$IN_COLAB

%env KAGGLE_DATASET_USERNAME = KAGGLE_DATASET_USERNAME
%env KAGGLE_DATASET_NAME = KAGGLE_DATASET_NAME

%env BUCKET_NAME=$BUCKET_NAME
!gcloud config set project $PROJECT_ID -q

env: IN_COLAB=False
env: KAGGLE_DATASET_USERNAME=KAGGLE_DATASET_USERNAME
env: KAGGLE_DATASET_NAME=KAGGLE_DATASET_NAME
env: BUCKET_NAME=aburdenko-llm-captstone
Updated property [core/project].


Upload Kaggle .json file

In [13]:
import sys
import os

FILE_EXISTS=os.path.exists( '/root/.kaggle/kaggle.json' )
if IN_COLAB and not FILE_EXISTS:

  !mkdir -p /root/.kaggle

  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    txt = "User uploaded file {name} with length {length} bytes".format( name=fn, length=len(uploaded[fn]))
    print( txt )

  import os
  os.rename(f"{fn}",'/root/.kaggle/kaggle.json')
  !chmod 600 /root/.kaggle/kaggle.json

  # import json
  # token = {"username":"alexburdenko","key":"3eb670cc106e7b8cfe41dfeaab51e3a2"}
  # with open('/root/.kaggle/kaggle.json', 'w') as file:
  #   json.dump(token, file)


!cat $HOME/.kaggle/kaggle.json


{"username":"alexburdenko","key":"bdc6f69f4aa714222d8c52fbf1aeca28"}

#Extract to GCP bucket

In [14]:
!kaggle competitions list

401 - Unauthorized


In [15]:
!kaggle datasets download $KAGGLE_DATASET_USERNAME/$KAGGLE_DATASET_NAME --force

401 - Unauthorized


In [16]:
from io import BytesIO
from zipfile import ZipFile
from zipfile import is_zipfile

text : str = ""

filename = f"{KAGGLE_DATASET_NAME}.zip"

if filename != ".zip":
  with open(filename, "rb") as fh:
      zipbytes = BytesIO(fh.read())

  if is_zipfile(zipbytes):
      with ZipFile(zipbytes, 'r') as myzip:
          for contentfilename in myzip.namelist():
              text = myzip.read(contentfilename)

print( text[:100] )

FileNotFoundError: [Errno 2] No such file or directory: 'medical-text.zip'

#Hit the Vertex AI PaLM Python API with an extractive Q&A task

In [ ]:
questions = [
    "Q: What is this document about?",
    "Q: What was the conclusion of this study?",
    "Q: Who was the subject of this study?",
    "Q: What hypothesis can you create about this study?",
    "Q: How would you test that hypothesis?"
]

str1 = "\n\n".join(questions)
text2 = (str(text[:1000])) + "\n\n" + str1
print(text2)


b'4\tCatheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior de

In [ ]:
from google.cloud.aiplatform.private_preview.language_models import TextGenerationModel

#input = '\n'.join(queries)
#input = text.join(input)
#print(input)

def extractive_question_answering_example(text, temperature=0 ):
    """Extractive Question Answering with a Large Language Model."""
    model = TextGenerationModel.from_pretrained("text-bison-001")
    response = model.predict( text,
      temperature=temperature,
      max_output_tokens=256,
      top_k=1,
      top_p=0,
    )
    txt : str = "\n" + str(response.text).replace( ". ", ".\n" )
    print(f"Response from Model: {txt}")

extractive_question_answering_example(text2)

Response from Model: 
1.
This document is about a study that assessed the safety of direct infarct angioplasty without antecedent thrombolytic therapy.
2.
The conclusion of this study was that direct angioplasty is a safe and effective treatment for acute myocardial infarction.
3.
The subjects of this study were patients with infarctions involving the left anterior descending, right, and circumflex coronary arteries.
4.
One hypothesis that could be created about this study is that patients who undergo direct angioplasty have a better outcome than patients who do not undergo direct angioplasty.
5.
One way to test this hypothesis would be to conduct a randomized controlled trial.


#Zero-Shot Prompts

In [ ]:
prompts = [
    "Prompt 1: “Tokenize the hashtags of the conversation",
    "Prompt 2: “create a bulleted list of UMLS codes that are mentioned in the conversation",
]

str1 = "\n\n".join(prompts)
text2 = (str(text[:1000])) + "\n\n" + str1
print(text2)

extractive_question_answering_example(text2)

b'4\tCatheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior de

#Few-shot prompts

In [ ]:
prompts = [
    "Prompt 1: “Decide whether the text's sentiment is positive, neutral, or negative.",
    "Prompt 2: “myocardial infarction is negative.",
    "Prompt 2: “sentiment:",
]

str1 = "\n\n".join(prompts)
text2 = (str(text[:1000])) + "\n\n" + str1
print(text2)

extractive_question_answering_example(text2)

b'4\tCatheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior de